# Importing the required libraries

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing the dataset

**These lines of code read in five CSV files into separate pandas dataframes.**


*   traindf1 is read from the file 'Train_Claim.csv'

*   traindf2 is read from the file 'Train_Demographics.csv'

*  traindf3 is read from the file 'Train_Policy.csv'
*   traindf4 is read from the file 'Train_Vehicle.csv'
*   traindf5 is read from the file 'Traindata_with_Target.csv

In [11]:
#loading the datset
traindf1 = pd.read_csv('/content/Train_Claim.csv')
traindf2 = pd.read_csv('/content/Train_Demographics.csv')
traindf3 = pd.read_csv('/content/Train_Policy.csv')
traindf4 = pd.read_csv('/content/Train_Vehicle.csv')
traindf5 = pd.read_csv('/content/Traindata_with_Target.csv')

ParserError: ignored

In [ ]:
## To display the first few rows of the dataframe:
traindf1

In [ ]:
## To view the first 5 rows and shape 
traindf1.head()
traindf1.shape

In [ ]:
## the number of missing values in each column of the traindf1 dataframe:
traindf1.isnull().sum()

In [ ]:
traindf2.head()
traindf2.shape

In [ ]:
traindf2.isnull().sum()

In [ ]:
display('traindf1', 'traindf2')

In [ ]:
traindf3.head()
traindf3.shape

In [ ]:
traindf3.isnull().sum()

In [ ]:
traindf4.head()
traindf4.shape

In [ ]:
traindf5.isnull().sum()

In [ ]:
traindf5.head()
traindf5.shape

## merging the train data

merges five datasets (traindf1, traindf2, traindf3, traindf4, and traindf5) on a common column named "CustomerID" using the inner join.

 The resulting merged_df dataframe contains only the rows where the CustomerID is present in all five datasets.

In [ ]:
# Merge the first two datasets on a common column
merged_df = pd.merge(traindf1, traindf2, on='CustomerID',how = 'inner')

# Merge the third dataset with the merged dataset on a common column
merged_df = pd.merge(merged_df, traindf3, on='CustomerID' , how = 'inner')

# Merge the fourth dataset with the merged dataset on a common column
merged_df = pd.merge(merged_df, traindf4, on='CustomerID', how = 'inner')

# Merge the fifth dataset with the merged dataset on a common column
merged_df = pd.merge(merged_df, traindf5, on='CustomerID', how ='inner')


In [ ]:
merged_df

# here the target column is ReportedFraud

In [ ]:
merged_df.shape

In [ ]:
## to get the data types of each column in the DataFrame.
merged_df.dtypes

In [ ]:
## summary of the DataFrame, including the number of non-null values and data types for each column.
merged_df.info()

provides descriptive statistics for the numerical columns in a DataFrame. It includes the count, mean, standard deviation, minimum value, 25th percentile, 50th percentile (median), 75th percentile, and maximum value.

In [ ]:
##Look at the statistical data
merged_df.describe()

# Preprocess Data

In [ ]:
merged_df.drop_duplicates(inplace=True)

whether each row is a duplicate or not. 

In [ ]:
## count the number of duplicates.
print(merged_df.duplicated().sum())

In [ ]:
## It looks like there are some missing values in the InsuredGender and Country columns.
merged_df.isnull().sum()

In [ ]:
print(merged_df.InsuredGender.value_counts())

In [ ]:
merged_df['InsuredGender'].fillna(merged_df['InsuredGender'].mode()[0], inplace=True)


In [ ]:
print(merged_df.Country.value_counts())

In [ ]:
merged_df['Country'].fillna(merged_df['Country'].mode()[0], inplace=True)

In [ ]:
merged_df.isnull().sum()

In [ ]:
print(merged_df.ReportedFraud.value_counts())

# Feature Engineering

In [ ]:
list_unique_columns=[] #list to store the unique columns
for i in merged_df.columns: ## each column name in the DataFrame merged_df
    if len(merged_df[i].value_counts())==1: ##checks if the number of unique values in the column i is equal to 1 
        list_unique_columns.append(i)
for i in list_unique_columns:
    merged_df.drop([i],axis=1,inplace=True)

In [ ]:
#replace ? with nan
merged_df=merged_df.replace('?',np.nan)

In [ ]:
# Check for question marks in dataframe
for column in merged_df.columns:
    print(column, merged_df[column][merged_df[column] == '?'].count())

In [ ]:
#missing value function
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100) ##o calculate the percentage of missing values in the dataframe. The code first sums the null values in the dataframe using isnull().sum() and then divides it by the total number of values in the dataframe (not null values) by using isnull().count(
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) ## concatenation two dataframes  total and percent
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt    

missing_data(merged_df)['Percent'].sort_values(ascending=False)

In [ ]:
merged_df['PropertyDamage'] = pd.to_numeric(merged_df['PropertyDamage'], errors='coerce')
merged_df['PoliceReport'] = pd.to_numeric(merged_df['PoliceReport'], errors='coerce')

In [ ]:
# fill missing values in 'PropertyDamage' and 'PoliceReport' columns with their mean values
merged_df['PropertyDamage'].fillna(merged_df['PropertyDamage'].mean(), inplace=True)
merged_df['PoliceReport'].fillna(merged_df['PoliceReport'].mean(), inplace=True)

In [ ]:
merged_df.drop('TypeOfCollission', axis=1, inplace=True)

# Numerical Features

In [ ]:
# Get numerical and categorical column names
numerical_cols = merged_df.select_dtypes(include='number').columns.tolist()
print("Numerical columns:", numerical_cols)


In [ ]:
# assuming 'df' is the pandas DataFrame containing the data
numerical_cols = ['IncidentTime', 'NumberOfVehicles', 'BodilyInjuries', 'AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'AmountOfVehicleDamage', 'InsuredAge', 'InsuredZipCode', 'CapitalGains', 'CapitalLoss', 'InsurancePolicyNumber', 'CustomerLoyaltyPeriod', 'Policy_Deductible', 'PolicyAnnualPremium', 'UmbrellaLimit']

unique_counts = merged_df[numerical_cols].nunique()
print(unique_counts)

In [ ]:
# Create box plots for each numerical column
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    plt.boxplot(merged_df[col])
    plt.title(col)
    plt.ylabel('Value')
    plt.show()

In [ ]:
# Create a box plot for each numeric column
merged_df.boxplot(column=list(merged_df.select_dtypes(include='number').columns))

# Calculate z-scores for all numerical variables
z_scores = merged_df.select_dtypes(include=['float64', 'int64']).apply(lambda x: (x - x.mean()) / x.std())

# Identify outliers based on z-scores greater than 3 or less than -3
outliers = (z_scores > 3) | (z_scores < -3)

# Print the number of outliers for each variable
print(outliers.sum())

In [ ]:
# Define a list of columns to replace outliers
cols_to_replace = ['AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'InsuredAge',
                   'CapitalLoss', 'PolicyAnnualPremium', 'UmbrellaLimit']

# Define a function to replace outliers with the median value
def replace_outliers(merged_df, col):
    q1 = merged_df[col].quantile(0.25)
    q3 = merged_df[col].quantile(0.75)
    iqr = q3 - q1
    lower_tail = q1 - 1.5 * iqr
    upper_tail = q3 + 1.5 * iqr
    med = np.median(merged_df[col])
    for i in merged_df[col]:
        if i > upper_tail or i < lower_tail:
            merged_df[col] = merged_df[col].replace(i, med)
    return merged_df

# Replace outliers for each column in the list
for col in cols_to_replace:
    merged_df = replace_outliers(merged_df, col)

# Plot the boxplots for each column after outlier treatment
fig, axs = plt.subplots(5, 2, figsize=(20, 30))
axs = axs.ravel()
for i, col in enumerate(cols_to_replace):
    sns.boxplot(x=col, data=merged_df, palette="Reds_r", ax=axs[i])
    axs[i].set_title("Box Plot for "+col+" after median imputation")
plt.show()

In [ ]:
# Group by 'InsuredGender' and create a histogram of 'PolicyAnnualPremium'
sns.histplot(data=merged_df, x='PolicyAnnualPremium', hue='InsuredGender', kde=True)
plt.show()

In [ ]:
merged_df.drop('UmbrellaLimit', axis=1, inplace=True)


In [ ]:
merged_df['NetCapitalGain'] = merged_df['CapitalGains'] - merged_df['CapitalLoss']


In [ ]:
import matplotlib.pyplot as plt

plt.hist(merged_df['NetCapitalGain'], bins=50)
plt.xlabel('Net Capital Gain')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.scatter(merged_df['PolicyAnnualPremium'], merged_df['NetCapitalGain'])
plt.xlabel('Policy Annual Premium')
plt.ylabel('Net Capital Gain')
plt.show()


In [ ]:
corr_coef = merged_df['PolicyAnnualPremium'].corr(merged_df['NetCapitalGain'])
print("Correlation coefficient between PolicyAnnualPremium and NetCapitalGain:", corr_coef)


create a scatter plot with a regression line that shows the relationship between 'PolicyAnnualPremium' and 'NetCapitalGain', as well as the correlation coefficient between the two variables.

In [ ]:
import seaborn as sns

corr_coef = merged_df['PolicyAnnualPremium'].corr(merged_df['NetCapitalGain'])  ## calculates the correlation coefficient between 'PolicyAnnualPremium' and 'NetCapitalGain' using the corr() method

sns.regplot(x='PolicyAnnualPremium', y='NetCapitalGain', data=merged_df, line_kws={'color': 'red'})## creates a scatter plot with a regression line using Seaborn's regplot() function. The x and y parameters specify the variables to be plotted, and the data parameter specifies the DataFrame to use for the plot. The line_kws parameter specifies the color of the regression line.
plt.xlabel('Policy Annual Premium') ## sets the x-axis label of the plot.
plt.ylabel('Net Capital Gain') ##sets the y-axis label of the plot.
plt.title('Correlation between Policy Annual Premium and Net Capital Gain (corr = {:.2f})'.format(corr_coef)) ##sets the title of the plot and includes the correlation coefficient value using string formatting.
plt.show()


# Categorical Features

In [ ]:
categorical_cols = merged_df.select_dtypes(include=['object', 'category']).columns.tolist()  ##selects all columns with data types that are either object or category. These are the data types typically used to represent categorical data.
print("Categorical columns:", categorical_cols)  ##prints the list of categorical column names.

In [ ]:
# Define the categorical columns
categorical_cols = ['CustomerID', 'DateOfIncident', 'TypeOfIncident', 
                    'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState', 'IncidentCity', 
                    'IncidentAddress', 'Witnesses', 
                    'InsuredGender', 'InsuredEducationLevel', 'InsuredOccupation', 
                    'InsuredHobbies', 'DateOfPolicyCoverage', 'InsurancePolicyState', 'Policy_CombinedSingleLimit', 
                    'InsuredRelationship', 'VehicleAttribute', 'VehicleAttributeDetails', 'ReportedFraud']

In [ ]:
columns_to_drop = ['CustomerID', 'DateOfIncident', 'IncidentAddress', 'Witnesses','InsuredHobbies','DateOfPolicyCoverage', 'Policy_CombinedSingleLimit']

merged_df = merged_df.drop(columns=columns_to_drop)

In [ ]:
print(merged_df.ReportedFraud.value_counts())

In [ ]:
# Count the number of values in the ReportedFraud column
fraud_counts = merged_df['ReportedFraud'].value_counts()

# Create a bar plot of the counts
plt.bar(fraud_counts.index, fraud_counts.values)

# Add labels and title
plt.xlabel('Reported Fraud')
plt.ylabel('Count')
plt.title('Number of Reported Fraud Cases')
plt.show()


In [ ]:
pd.crosstab(merged_df.InsuredGender, merged_df.ReportedFraud).plot(kind='bar')
plt.title('Number of Reported Fraud Cases')

In [ ]:
# Select categorical columns to plot
cat_cols = ['TypeOfIncident', 'SeverityOfIncident', 'AuthoritiesContacted', 'InsuredGender', 'InsuredEducationLevel']

# Plot pie chart for each categorical column
for col in cat_cols:
    plt.figure()
    merged_df[col].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title(col)
    plt.axis('equal')
    plt.show()


In [ ]:
# Define a dictionary to map category values to numerical labels
label_map = {} ## We create an empty dictionary to store the names of the labels
label_count = 0 ## the label count initialized to 0.


for col in categorical_cols: ##for every column (col) in the categorical column (categorical_cols) list
    
    # If the column is not in the columns to drop list
    if col not in columns_to_drop:
        
        # Get unique values in the column
        unique_vals = merged_df[col].unique()
        
        # Loop through each unique value
        for val in unique_vals:
            
            # If the value has not been seen before, assign a new label
            if val not in label_map:
                label_map[val] = label_count
                label_count += 1
                
        # Replace categorical values in the column with their corresponding numerical labels
        merged_df[col] = merged_df[col].apply(lambda x: label_map[x]) 

## loading the test data

In [ ]:
testdf1 = pd.read_csv('/content/Test.csv')
testdf2 = pd.read_csv('/content/Test_Claim.csv')
testdf3 = pd.read_csv('/content/Test_Demographics.csv')
testdf4 = pd.read_csv('/content/Test_Policy.csv')
testdf5 = pd.read_csv('/content/Test_Vehicle.csv')

In [ ]:
testdf1.head()

In [ ]:
testdf1.shape

In [ ]:
testdf2.shape

In [ ]:
testdf3.shape

In [ ]:
testdf4.shape

In [ ]:
testdf5.shape

## Merging the test data

In [ ]:
# Merge the first two datasets on a common column
merged_test_df = pd.merge(testdf1, testdf2, on='CustomerID')

# Merge the third dataset with the merged dataset on a common column
merged_test_df = pd.merge(merged_test_df, testdf3, on='CustomerID')

# Merge the fourth dataset with the merged dataset on a common column
merged_test_df = pd.merge(merged_test_df, testdf4, on='CustomerID')

# Merge the fifth dataset with the merged dataset on a common column
merged_test_df = pd.merge(merged_test_df, testdf5, on='CustomerID')

In [ ]:
merged_test_df.head()

In [ ]:
merged_test_df.shape

In [ ]:
merged_test_df.drop_duplicates(inplace=True)

In [ ]:
merged_test_df.isnull().sum()

In [ ]:
print(merged_test_df.InsuredGender.value_counts())

In [ ]:
merged_test_df['InsuredGender'].fillna(merged_test_df['InsuredGender'].mode()[0], inplace=True)

In [ ]:
merged_test_df['Country'].fillna(merged_test_df['Country'].mode()[0], inplace=True)

In [ ]:
merged_test_df.isnull().sum()

In [ ]:
list_unique_columns=[]
for i in merged_test_df.columns:
    if len(merged_test_df[i].value_counts())==1:
        list_unique_columns.append(i)
for i in list_unique_columns:
    merged_test_df.drop([i],axis=1,inplace=True)

In [ ]:
#replace ? with nan
merged_test_df=merged_test_df.replace('?',np.nan)

In [ ]:
for column in merged_test_df.columns:
    print(column, merged_test_df[column][merged_test_df[column] == '?'].count())

In [ ]:
#missing value function
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt    

missing_data(merged_test_df)['Percent'].sort_values(ascending=False)

In [ ]:
merged_test_df['PropertyDamage'].fillna(merged_test_df['PropertyDamage'].mode()[0], inplace=True)

In [ ]:
merged_test_df['PoliceReport'].fillna(merged_test_df['PoliceReport'].mode()[0], inplace=True)

In [ ]:
merged_test_df['TypeOfCollission'].fillna(merged_test_df['TypeOfCollission'].mode()[0], inplace=True)

In [ ]:
#missing value function
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return tt    

missing_data(merged_test_df)['Percent'].sort_values(ascending=False)

In [ ]:
column_names = list(merged_test_df.columns)
print(column_names)

In [ ]:
# Get numerical and categorical column names
numerical_cols = merged_test_df.select_dtypes(include='number').columns.tolist()
print("Numerical columns:", numerical_cols)

In [ ]:
# Create box plots for each numerical column
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    plt.boxplot(merged_test_df[col])
    plt.title(col)
    plt.ylabel('Value')
    plt.show()

In [ ]:
# Define a list of columns to replace outliers
cols_to_replace = ['AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'InsuredAge',
                   'CapitalLoss', 'PolicyAnnualPremium', 'UmbrellaLimit']

# Define a function to replace outliers with the median value
def replace_outliers(merged_test_df, col):
    q1 = merged_test_df[col].quantile(0.25)
    q3 = merged_test_df[col].quantile(0.75)
    iqr = q3 - q1
    lower_tail = q1 - 1.5 * iqr
    upper_tail = q3 + 1.5 * iqr
    med = np.median(merged_test_df[col])
    for i in merged_test_df[col]:
        if i > upper_tail or i < lower_tail:
            merged_test_df[col] = merged_test_df[col].replace(i, med)
    return merged_test_df

# Replace outliers for each column in the list
for col in cols_to_replace:
    merged_test_df = replace_outliers(merged_test_df, col)

# Plot the boxplots for each column after outlier treatment
fig, axs = plt.subplots(5, 2, figsize=(20, 30))
axs = axs.ravel()
for i, col in enumerate(cols_to_replace):
    sns.boxplot(x=col, data=merged_test_df, palette="Reds_r", ax=axs[i])
    axs[i].set_title("Box Plot for "+col+" after median imputation")
plt.show()

In [ ]:
# assuming 'df' is the pandas DataFrame containing the data
numerical_cols = ['IncidentTime', 'NumberOfVehicles', 'BodilyInjuries', 'AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'AmountOfVehicleDamage', 'InsuredAge', 'InsuredZipCode', 'CapitalGains', 'CapitalLoss', 'InsurancePolicyNumber', 'CustomerLoyaltyPeriod', 'Policy_Deductible', 'PolicyAnnualPremium', 'UmbrellaLimit']

unique_counts = merged_test_df[numerical_cols].nunique()
print(unique_counts)

In [ ]:
categorical_cols = merged_test_df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical columns:", categorical_cols)

In [ ]:
# Define the categorical columns
categorical_cols = ['CustomerID', 'DateOfIncident', 'TypeOfIncident', 
                    'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState', 'IncidentCity', 
                    'IncidentAddress', 'Witnesses', 
                    'InsuredGender', 'InsuredEducationLevel', 'InsuredOccupation', 
                    'InsuredHobbies', 'DateOfPolicyCoverage', 'InsurancePolicyState', 'Policy_CombinedSingleLimit', 
                    'InsuredRelationship', 'VehicleAttribute', 'VehicleAttributeDetails']


In [ ]:
columns_to_drop = ['CustomerID', 'DateOfIncident', 'IncidentAddress', 'Witnesses', 'PoliceReport',  'InsuredHobbies', 'DateOfPolicyCoverage', 'Policy_CombinedSingleLimit']

merged_test_df = merged_test_df.drop(columns=columns_to_drop)

In [ ]:
# Define a dictionary to map category values to numerical labels
label_map = {}
label_count = 0

# Loop through each categorical column
for col in categorical_cols:
    
    # If the column is not in the columns to drop list
    if col not in columns_to_drop:
        
        # Get unique values in the column
        unique_vals = merged_test_df[col].unique()
        
        # Loop through each unique value
        for val in unique_vals:
            
            # If the value has not been seen before, assign a new label
            if val not in label_map:
                label_map[val] = label_count
                label_count += 1
                
        # Replace categorical values in the column with their corresponding numerical labels
        merged_test_df[col] = merged_test_df[col].apply(lambda x: label_map[x])